In [1]:
import numpy as np 
import pandas as pd 
import talib 
import yfinance as yf
from datetime import datetime
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt 
import statistics 


class parentTSClass:
    def __init__(self, target_stock, start_date, end_date):
        self.target_stock = target_stock
        self.start_date = start_date
        self.end_date = end_date

class maTSClass(parentTSClass):
    def __init__(self, target_stock, start_date, end_date, stopLoss):
        super().__init__(target_stock, start_date, end_date)  # Corrected inheritance initialization
        self.stopLoss = stopLoss

    def tradingStrategy(self):
        target_stock = self.target_stock
        start_date = self.start_date 
        end_date = self.end_date 
        stopLoss = self.stopLoss 

        yf.pdr_override() 
        df = data.get_data_yahoo([target_stock], start_date, end_date) 
        df = df.reindex(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
        df.rename(columns={'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True) 
        self.drawStockSeries(df, target_stock)  # Correct placement of the method call

        closePrices = df['close'].astype('float').values
        close_sma_5 = np.round(talib.SMA(closePrices, timeperiod=5), 2) 
        close_sma_10 = np.round(talib.SMA(closePrices, timeperiod=10), 2) 
        close_sma_20 = np.round(talib.SMA(closePrices, timeperiod=20), 2) 
        self.drawMA_5_10_20Series(df.index, close_sma_5, close_sma_10, close_sma_20, target_stock)  # Correct method call

        flage = 0 
        buyPrice = 0 
        sellPrice = 0 
        winTime = 0 
        lossTime = 0 
        culReturn = 0 
        transList = [] 
        everyTranReturn = [] 
        tradingDetails = [] 
        tax = 0 

        for x in range(19, len(closePrices)):
            if flage == 0: 
                if close_sma_5[x] > close_sma_10[x] and close_sma_10[x] > close_sma_20[x]: 
                    buyPrice = closePrices[x]
                    tradingDetails.append(("買進日期 =", str(df.index[x])[:11], "買進價格 =", format(buyPrice, ".2f")))
                    tax += buyPrice * 0.001425
                    flage = 1

            if flage == 1: 
                sellPrice = closePrices[x]

                if close_sma_5[x] < close_sma_10[x] and close_sma_10[x] < close_sma_20[x]: 
                    tax += sellPrice * 0.001425
                    tradingDetails.append(("賣出日期 =", str(df.index[x])[:11], "賣出價格 =",  "賺", format((sellPrice - buyPrice - tax), ".2f"), 
                                           format((sellPrice - buyPrice - tax) / buyPrice, ".2%")))

                    if (sellPrice - buyPrice - tax) / buyPrice < -stopLoss: 
                        lossTime += 1
                        flage = 0
                        everyTranReturn.append((sellPrice - buyPrice) - tax) 
                        culReturn += (sellPrice - buyPrice) - tax
                        transList.append(culReturn) 
                        tax = 0 
                        self.drawCumulativeReturnSeries(transList)  

        return culReturn
# draw Moving average graphs
    def drawStockSeries(self, df, target_stock):
        plt.figure(figsize=(14, 7))
        plt.plot(df['close'], label=f'{target_stock} Close Price')
        plt.title(f'{target_stock} Stock Price')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

    def drawMA_5_10_20Series(self, dates, close_sma_5, close_sma_10, close_sma_20, target_stock):
        plt.figure(figsize=(14, 7))
        plt.plot(dates, close_sma_5, label='SMA 5')
        plt.plot(dates, close_sma_10, label='SMA 10')
        plt.plot(dates, close_sma_20, label='SMA 20')
        plt.title(f'{target_stock} Moving Averages')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

    def drawCumulativeReturnSeries(self, transList):
        plt.figure(figsize=(14, 7))
        plt.plot(transList, label='Cumulative Return')
        plt.title('Cumulative Return Over Time')
        plt.xlabel('Transaction')
        plt.ylabel('Cumulative Return')
        plt.legend()
        plt.show()
        
# use the MA Class for other parts (for example: to test out and combine with other programs like ROC for example.) 